In [21]:
import pandas as pd
import folium

# 디스플레이 옵션 설정
pd.set_option('display.width', None)        # 출력화면의 너비
pd.set_option('display.max_rows', 100)      # 출력할 행의 개수 한도
pd.set_option('display.max_columns', 10)    # 출력할 열의 개수 한도
pd.set_option('display.max_colwidth', 20)   # 출력할 열의 너비
pd.set_option('display.unicode.east_asian_width', True)   # 유니코드 사용 너비 조정


'''
[Step 1] 데이터 준비
'''

# 서울시내 중학교 진학률 데이터셋
file_path = './data/middle_shcool_graduates_report.xlsx'
df = pd.read_excel(file_path)

# 열 이름 배열을 출력
print(df.columns.values)

df.head() 

# 데이터 자료형 확인
df.info()

df.describe()

df.isnull().sum().sum()

df.duplicated().sum()

attr = (
    'Map data: &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors, <a href="http://viewfinderpanoramas.org">SRTM</a> | Map style: &copy; <a href="https://opentopomap.org">OpenTopoMap</a> (<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC-BY-SA</a>)'
)

tiles = 'https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png'

mschool_map = folium.Map(location=[37.55,126.98], tiles=tiles, attr=attr, 
                         zoom_start=12)

# 중학교 위치정보를 CircleMarker로 표시
for name, lat, lng in zip(df['학교명'], df['위도'], df['경도']):
    folium.CircleMarker([lat, lng],
                        radius=5,              # 원의 반지름
                        color='brown',         # 원의 둘레 색상
                        fill=True,
                        fill_color='coral',    # 원을 채우는 색
                        fill_opacity=0.7,      # 투명도    
                        popup=name
    ).add_to(mschool_map)
    
mschool_map   

mschool_map.save('./data/seoul_mschool_location.html')

print(df['지역'].nunique())
print(df['코드'].nunique())
print(df['유형'].nunique())
print(df['주야'].nunique())

df_encoded = pd.get_dummies(df, columns=['지역', '코드', '유형', '주야'])

df_encoded.head()

from sklearn import cluster
from sklearn import preprocessing  

# 분석에 사용할 속성을 선택 
train_features = ['과학고', '외고_국제고', '자사고', '자공고', 
                  '유형_공립', '유형_국립', '유형_사립',]
X = df_encoded.loc[:, train_features]

# 설명 변수 데이터를 정규화
X = preprocessing.StandardScaler().fit_transform(X)

# DBSCAN 모형 객체 생성
dbm = cluster.DBSCAN(eps=0.2, min_samples=5)

# 모형 학습
dbm.fit(X)   
 
# 예측 (군집) 
cluster_label = dbm.labels_   
print(cluster_label)

df_encoded['Cluster'] = cluster_label
df_encoded.head()

grouped_cols = ['학교명', '과학고', '외고_국제고', '자사고',] 
grouped = df_encoded.groupby('Cluster')
for key, group in grouped:
    print('* key :', key)
    print('* number :', len(group))    
    print(group.loc[:, grouped_cols].head())
    print('\n')

colors = {-1:'gray', 0:'coral', 1:'blue', 2:'green', 3:'red', 4:'purple', 
          5:'orange', 6:'brown', 7:'brick', 8:'yellow', 9:'magenta', 10:'cyan', 11:'tan'}

cluster_map = folium.Map(location=[37.55,126.98], tiles=tiles, attr=attr, 
                         zoom_start=12)

for name, lat, lng, clus in zip(df_encoded['학교명'], df_encoded['위도'], 
                                df_encoded['경도'], df_encoded['Cluster']):   
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[clus],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[clus],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster_map)

cluster_map

cluster_map.save('./data/seoul_mschool_cluster.html')

['지역' '학교명' '코드' '유형' '주야' '남학생수' '여학생수' '일반고' '특성화고' '과학고' '외고_국제고'
 '예고_체고' '마이스터고' '자사고' '자공고' '기타진학' '취업' '미상' '위도' '경도']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역      415 non-null    object 
 1   학교명     415 non-null    object 
 2   코드      415 non-null    int64  
 3   유형      415 non-null    object 
 4   주야      415 non-null    object 
 5   남학생수    415 non-null    int64  
 6   여학생수    415 non-null    int64  
 7   일반고     415 non-null    float64
 8   특성화고    415 non-null    float64
 9   과학고     415 non-null    float64
 10  외고_국제고  415 non-null    float64
 11  예고_체고   415 non-null    float64
 12  마이스터고   415 non-null    float64
 13  자사고     415 non-null    float64
 14  자공고     415 non-null    float64
 15  기타진학    415 non-null    float64
 16  취업      415 non-null    int64  
 17  미상      415 non-null    float64
 18  위도      415 non-null    